In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup
import time

from datetime import datetime
from datetime import timedelta
from dateutil import relativedelta

In [3]:
# driver = webdriver.Chrome()
# driver.maximize_window()
# driver.get("https://music.bugs.co.kr/album/20332694")
# html = driver.page_source # 드라이버 현재 페이지의 html 정보 가져오기 
#                             # cf) requests.get(url)
# soup = BeautifulSoup(html, 'lxml')

# soup.find_all('td')[2].text.strip('\n')
# # [release.find('time')['datetime'] for release in soup.find_all('table', attrs={'class' : 'info'})]


In [11]:
search_year = 2020
search_month = 8

for j in range(1):
        search_month += j
        if search_month > 12:
            search_month -= 12
        
        if search_month >= 10:
            search_url = "https://circlechart.kr/page_chart/onoff.circle?nationGbn=T&serviceGbn=ALL&targetTime="+str(search_month)+"&hitYear="+str(search_year)+"&termGbn=month&yearTime=3"

        else:
            search_url = "https://circlechart.kr/page_chart/onoff.circle?nationGbn=T&serviceGbn=ALL&targetTime=0"+str(search_month)+"&hitYear="+str(search_year)+"&termGbn=month&yearTime=3"
            
        date = datetime(search_year, search_month, 1, 0, 0, 0)
        date = date + relativedelta.relativedelta(months=1)
        
        thismonth_lastday = date - timedelta(seconds=1)
        
        date_list = [ thismonth_lastday for i in range(200)]
        
        driver = webdriver.Chrome()
        driver.maximize_window()
        driver.get(search_url)
        html = driver.page_source # 드라이버 현재 페이지의 html 정보 가져오기 
                            # cf) requests.get(url)
        soup = BeautifulSoup(html, 'lxml')
        
        
        title = [title.get_text() for title in soup.find_all('div', attrs={'class': 'font-bold mb-2'})]
        rank = [rank.get_text() for rank in soup.find_all('span', attrs={'class' : 'text-2xl'})]
        artist = [artist.get_text().split('|')[0] for artist in soup.find_all('div', attrs={'class' : 'text-sm text-gray-400 font-bold'})]
        score = [score.get_text() for score in soup.find_all('span' ,attrs={'style' : "font-family: system-ui;"})]
        rank_change = [rank_change.find('span').get_text() for rank_change in soup.find_all('div', attrs={'class' : "h-full text-center leading-[30px] mt-[7px] pl-[10px]"})]
        
        
        release_date = []
        genre = []

        data_seqmom_values = [div['data-seqmom'] for div in soup.find_all('div', attrs= {'class' : 'playBtnDiv'})]

        album_values = []
        for i in data_seqmom_values:
            album_values.append("PlayLogoOnclick('1594',"+"'"+i+"'"+")")


        for i in album_values:

            driver.execute_script(i)
            
            driver.switch_to.window(driver.window_handles[-1])

            time.sleep(2)
#             wait = WebDriverWait(driver, 10)
#             wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table.info")))
            
            html = driver.page_source # 드라이버 현재 페이지의 html 정보 가져오기 

            soup = BeautifulSoup(html, 'lxml')
    
            release_date.append(soup.find_all('td')[2].text.strip('\n'))
            genre.append(soup.find_all('td')[3].text.strip('\n'))
            
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            
        data = {"날짜": date_list, "순위": rank, "곡명": title, "가수":artist, "score": score, "순위 변화":rank_change, "장르" : genre, "발매일" : release_date}
        df = pd.DataFrame(data)
        if search_month >= 10:
            df.to_csv(str(search_year)+"_"+str(search_month)+"_.csv", encoding="utf-8-sig")
        else:
            df.to_csv(str(search_year)+"_0"+str(search_month)+"_.csv", encoding="utf-8-sig")